# Performance Analysis Notebook 

In [22]:
import json
import pandas as pd
from IPython.display import display


with open("logs/training_log.json") as f:
    logs = json.load(f)

rows = []

for entry in logs:
    timestamp = entry.get("timestamp")
    dataset = entry.get("dataset")
    models = entry.get("models", [])

    for model_name in models:
        model_block = entry.get(model_name, None)
        if model_block is None:
            continue  # skip if missing (older logs etc.)

        hp = model_block.get("hyperparameters", {})
        cv = model_block.get("cv_metrics", {})

        row = {
            "timestamp": timestamp,
            "dataset": dataset,
            "model": model_name,
            "val_acc": cv.get("pico_val_acc"),
            "val_loss": cv.get("pico_val_loss"),
            "epochs_trained": cv.get("epochs_trained"),
            "test_acc": model_block.get("pico_accuracy"),
            "mean_val_acc": cv.get("mean_val_acc"),
            "std_val_acc": cv.get("std_val_acc"),
        }

        # Add all hyperparameters with prefix "hp_"
        for k, v in hp.items():
            row[f"hp_{k}"] = v

        rows.append(row)

df = pd.DataFrame(rows)
cols = ["model", "hp_win", "hp_hop", "hp_batch_size", "hp_epochs", "hp_lr", "hp_dropout", "hp_augment", "val_acc", "val_loss", "epochs_trained", "test_acc"]
df = pd.DataFrame(rows)

renamed = df.rename(columns={
    "hp_win": "Window Size",
    "hp_hop": "Hop Size",
    "hp_batch_size": "Batch Size",
    "hp_epochs": "Max Epochs",
    "hp_lr": "Learning Rate",
    "hp_dropout": "Dropout",
    "hp_augment": "Augmentation",
    "val_acc": "Validation Accuracy",
    "val_loss": "Validation Loss",
    "mean_val_acc": "Mean Validation Accuracy",
    "std_val_acc": "Std Validation Accuracy",
    "epochs_trained": "Epochs Trained",
    "test_acc": "Test Accuracy",
})

cols = [
    "model",
    "Mean Validation Accuracy",
    "Std Validation Accuracy",
    "Window Size",
    "Hop Size",
    "Batch Size",
    "Max Epochs",
    "Learning Rate",
    "Dropout",
    "Augmentation",
    "Validation Accuracy",
    "Validation Loss",
    "Epochs Trained",
    "Test Accuracy",
]

# Avoid NaNs at top of accuracy sorted list 
df_sorted = renamed.sort_values(
    by="Mean Validation Accuracy",
    ascending=False,
    na_position="last"
)

# build Styler with background gradients
styled = (
    df_sorted[cols]
    .style
    # accuracies: higher = better
    .background_gradient(
        cmap="Greens",
        subset=[c for c in ["Validation Accuracy", "Mean Validation Accuracy", "Test Accuracy"]
                if c in df_sorted.columns],
    )
    # std: lower = better (reverse Reds)
    .background_gradient(
        cmap="Reds_r",
        subset=[c for c in ["Std Validation Accuracy"] if c in df_sorted.columns],
    )
    # learning rate
    .background_gradient(
        cmap="Blues",
        subset=[c for c in ["Learning Rate"] if c in df_sorted.columns],
    )
    # epochs
    .background_gradient(
        cmap="Purples",
        subset=[c for c in ["Epochs Trained"] if c in df_sorted.columns],
    )
    .format({
        "Validation Accuracy": "{:.4f}",
        "Mean Validation Accuracy": "{:.4f}",
        "Std Validation Accuracy": "{:.4f}",
        "Validation Loss": "{:.4f}",
        "Test Accuracy": "{:.4f}",
        "Learning Rate": "{:.1e}",
    })
)

display(styled)


,model,Mean Validation Accuracy,Std Validation Accuracy,Window Size,Hop Size,Batch Size,Max Epochs,Learning Rate,Dropout,Augmentation,Validation Accuracy,Validation Loss,Epochs Trained,Test Accuracy
30,one_d_cnn,0.8501,0.0793,64,64,16,1200,1.0e-04,0.250000,True,nan,nan,nan,nan
23,one_d_cnn,0.8432,0.0706,64,64,16,1200,1.0e-04,0.250000,True,nan,nan,nan,nan
6,one_d_cnn,0.8295,0.0804,64,64,16,800,1.0e-04,0.300000,True,nan,nan,nan,0.3314
17,one_d_cnn,0.8113,0.0662,64,64,64,800,5.0e-04,0.300000,True,nan,nan,nan,nan
12,one_d_cnn,0.7870,0.1198,64,64,32,500,5.0e-05,0.200000,True,nan,nan,nan,0.3314
16,one_d_cnn,0.7789,0.1238,64,64,32,800,1.0e-04,0.400000,True,nan,nan,nan,nan
2,one_d_cnn,0.7783,0.1540,64,64,16,200,1.0e-04,0.300000,True,nan,nan,nan,0.3314
18,one_d_cnn,0.7637,0.1326,64,64,64,800,5.0e-04,0.400000,True,nan,nan,nan,nan
0,cnn,0.7592,0.0972,64,64,32,200,4.0e-04,0.300000,False,nan,nan,nan,0.3314
13,cnn,0.7540,0.1229,64,64,32,500,1.0e-04,0.300000,False,nan,nan,nan,0.3314
